In [ ]:
#hide
from tweet_archiveur.scrapper import *

None


# Tweet Archiveur

> This project aim at storing the tweet of all members of the French Parliament.

The goal is to use tweets to get an idea of the topics of the tweets using NLP.

## Install

(TODO : push it to Pipy)

`pip install tweetarchiveur`

## How to use

Fill me in please! Don't forget code examples:

## What to do with it ?

- Most used hashtag (per period, per person)
- Most/Less active user
- Timeline of 
- NLP Topic detection
- Word cloud

# Annexes

If one thing fail no tweet will be saved.

status code = 429 : 429 'Too many requests' error is returned when you exceed the maximum number of requests allowed 